In [1]:
import os
import tensorflow as tf
import time

In [2]:
def decode(serialized_example):
    features = tf.parse_single_example(
      serialized_example,      
      features={
          'label': tf.FixedLenFeature([], tf.int64),
          'width': tf.FixedLenFeature([], tf.int64),
          'height': tf.FixedLenFeature([], tf.int64),
          'image': tf.FixedLenFeature([], tf.string)
      })
    image = tf.decode_raw(features['image'],tf.uint8)
    #image = tf.reshape(image,[64,64,1])
    image = tf.cast(image, tf.float32)
    image = tf.reshape(image, [64,64,1])
    label = tf.cast(features['label'],tf.int64)
    
    return image,label

In [3]:
batch_size=100
train_file_name = "Shuffled_PE92_Train.tfrecord"
train_dataset = tf.data.TFRecordDataset("./"+train_file_name)
train_dataset = train_dataset.map(decode)
train_dataset = train_dataset.repeat()
train_dataset= train_dataset.shuffle(buffer_size=10000)
train_dataset = train_dataset.batch(batch_size)


In [4]:
test_file_name = "Shuffled_PE92_Test.tfrecord"
test_dataset = tf.data.TFRecordDataset("./"+test_file_name)
test_dataset = test_dataset.map(decode)
test_dataset = test_dataset.repeat()
test_dataset= test_dataset.shuffle(buffer_size=100)
test_dataset = test_dataset.batch(batch_size)

In [5]:
train_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+train_file_name))
print(train_data_length)

test_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+test_file_name))

200215


In [6]:
train_iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
test_iterator = tf.data.Iterator.from_structure(test_dataset.output_types,
                                           test_dataset.output_shapes)

train_image, train_label = train_iterator.get_next()
test_image, test_label = train_iterator.get_next()

train_init_op = train_iterator.make_initializer(train_dataset)
test_init_op = test_iterator.make_initializer(test_dataset)

train_total_batch = int(train_data_length/batch_size)
test_total_batch = int(test_data_length/batch_size)

In [7]:
input_images = tf.placeholder(tf.float32,[None,64,64,1])
input_labels = tf.placeholder(tf.int64,[None])
is_training = tf.placeholder(tf.bool)
keep_prob = tf.placeholder(tf.float32)
print(input_images)
print(input_labels)

Tensor("Placeholder:0", shape=(?, 64, 64, 1), dtype=float32)
Tensor("Placeholder_1:0", shape=(?,), dtype=int64)


In [8]:
W1 = tf.Variable(tf.random_normal([5,5,1,32], stddev=0.01))
L1 = tf.nn.conv2d(input_images,W1, strides=[1,1,1,1], padding = 'SAME')
print(L1)
bn1 = tf.layers.batch_normalization(L1, training=is_training)
at1 = tf.nn.relu(bn1)

pool1 =  tf.nn.max_pool(at1, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool1)

Tensor("Conv2D:0", shape=(?, 64, 64, 32), dtype=float32)
Tensor("MaxPool:0", shape=(?, 32, 32, 32), dtype=float32)


In [9]:
W2 = tf.Variable(tf.random_normal([5, 5, 32, 64], stddev=0.01))

L2 = tf.nn.conv2d(pool1,W2, strides=[1,1,1,1], padding = 'SAME')
bn2 = tf.layers.batch_normalization(L2, training=is_training)
at2 = tf.nn.relu(bn2)

pool2 =  tf.nn.max_pool(at2, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool2)

Tensor("MaxPool_1:0", shape=(?, 16, 16, 64), dtype=float32)


In [10]:
W3 = tf.Variable(tf.random_normal([4, 4, 64, 128], stddev=0.01))

L3 = tf.nn.conv2d(pool2,W3, strides=[1,1,1,1], padding = 'SAME')
bn3 = tf.layers.batch_normalization(L3, training=is_training)
at3 = tf.nn.relu(bn3)

pool3 =  tf.nn.max_pool(at3, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool3)

Tensor("MaxPool_2:0", shape=(?, 8, 8, 128), dtype=float32)


In [11]:
W4 = tf.Variable(tf.random_normal([4, 4, 128, 256], stddev=0.01))

L4 = tf.nn.conv2d(pool3,W4, strides=[1,1,1,1], padding = 'SAME')
bn4 = tf.layers.batch_normalization(L4, training=is_training)
at4 = tf.nn.relu(bn4)

pool4 =  tf.nn.max_pool(at4, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool4)

Tensor("MaxPool_3:0", shape=(?, 4, 4, 256), dtype=float32)


In [12]:
W5 = tf.Variable(tf.random_normal([4*4*256, 1024], stddev=0.01))
fc1 = tf.reshape(pool4,[-1,4*4*256])
fc1 = tf.matmul(fc1, W5)
fc1 = tf.nn.relu(fc1)

fc1 = tf.nn.dropout(fc1,keep_prob)
print(fc1)

Tensor("dropout/mul:0", shape=(?, 1024), dtype=float32)


In [13]:
W6 = tf.Variable(tf.random_normal([1024,2350],stddev=0.01))
logit = tf.matmul(fc1, W6)
print(logit)

Tensor("MatMul_1:0", shape=(?, 2350), dtype=float32)


In [14]:
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logit, labels=input_labels))
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)
print(loss)

Tensor("Mean:0", shape=(), dtype=float32)


In [15]:
correct_prediction = tf.equal(tf.argmax(logit,1),input_labels)

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
saver = tf.train.Saver()

print("Train total Batch :%d" % (train_total_batch))
with tf.Session() as sess:
    sess.run(init)
    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    #Train Model
    loss_sum=0
    start_time = time.time()
    for epoch in range(50):
        sess.run(train_init_op)
        for i in range(train_total_batch):
        #for i in range(batch_size):
            batch_images, batch_labels = sess.run([train_image, train_label])
            sess.run(optimizer,feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:0.5, is_training:True})
            train_acc =sess.run(accuracy, feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:1.0,is_training:False})
            
            if(i%100==0):
                print('Epoch :%d | step %d, training accuracy: %f' % (epoch,i, train_acc))

        save_path = saver.save(sess, "save\model_PE92.ckpt")
        if(epoch%5==0):
            sess.run(test_init_op)#Test iterator 생성
            batch_images, batch_labels = sess.run([test_image, test_label])
            test_acc, train_loss=sess.run([accuracy, loss], feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:1.0, is_training:False})
            print("Test Acc : %f" %(test_acc))
            
    end_time = time.time()
    elapsed = end_time - start_time
    
    print("Training is done.")
    print("Accuracy :%f" %(train_acc))
    print("Time : %f" %(elapsed))
    
    #Test Model
    coord.request_stop()
    coord.join(threads)

Train total Batch :2002
Epoch :0 | step 0, training accuracy: 0.020000
Epoch :0 | step 100, training accuracy: 0.000000
Epoch :0 | step 200, training accuracy: 0.000000
Epoch :0 | step 300, training accuracy: 0.000000
Epoch :0 | step 400, training accuracy: 0.000000
Epoch :0 | step 500, training accuracy: 0.000000
Epoch :0 | step 600, training accuracy: 0.000000
Epoch :0 | step 700, training accuracy: 0.000000
Epoch :0 | step 800, training accuracy: 0.000000
Epoch :0 | step 900, training accuracy: 0.010000
Epoch :0 | step 1000, training accuracy: 0.000000
Epoch :0 | step 1100, training accuracy: 0.010000
Epoch :0 | step 1200, training accuracy: 0.000000
Epoch :0 | step 1300, training accuracy: 0.000000
Epoch :0 | step 1400, training accuracy: 0.000000
Epoch :0 | step 1500, training accuracy: 0.000000
Epoch :0 | step 1600, training accuracy: 0.000000
Epoch :0 | step 1700, training accuracy: 0.000000
Epoch :0 | step 1800, training accuracy: 0.000000
Epoch :0 | step 1900, training accurac

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    sess.run(train_init_op)
    start_time = time.time()
    for i in range(train_total_batch):
        #for i in range(batch_size):
        batch_images, batch_labels = sess.run([train_image, train_label])
        print(batch_labels)

        
    coord.request_stop()
    coord.join(threads)

In [1]:
import tensorflow as tf
from random import randint

dims = 8
pos  = randint(0, dims - 1)
print(pos)
logits = tf.random_uniform([dims], maxval=3, dtype=tf.float32)
labels = tf.one_hot(pos, dims)
print(logits)
print(labels)

res1 = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels)
res2 = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=tf.constant(pos))
loss=tf.reduce_mean(res2)
with tf.Session() as sess:
    a, b = sess.run([res1, loss])
    print (a, b)
    print (a == b)

4
Tensor("random_uniform:0", shape=(8,), dtype=float32)
Tensor("one_hot:0", shape=(8,), dtype=float32)
1.1221625 1.1221625
True


In [ ]:
def read_and_decode(filename_queue, batch_size):
    tf_record_reader=tf.TFRecordReader()
    _, tf_record_serialized = tf_record_reader.read(tf_record_filename_queue)

    tf_record_features = tf.parse_single_example(
        tf_record_serialized,
        features={
            'label': tf.FixedLenFeature([], tf.int64),
            'width': tf.FixedLenFeature([], tf.int64),
            'height': tf.FixedLenFeature([], tf.int64),
            'image': tf.FixedLenFeature([], tf.string),
        })
    tf_record_image = tf.decode_raw(tf_record_features['image'],tf.uint8)
    
    tf_record_image = tf.cast(tf_record_image, tf.float32)

    tf_record_image = tf.reshape(tf_record_image, [64,64,1])

    tf_record_label = tf.cast(tf_record_features['label'],tf.int64)
    
    images, labels = tf.train.shuffle_batch([tf_record_image, tf_record_label],
                                         batch_size = batch_size,
                                         capacity=20*batch_size,
                                         min_after_dequeue=5*batch_size )
    return images, labels